# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [ ]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # lowercase, remove special characters, punctuations, numbers. Also tokenizes the text into a list of words.
            # The yield statement turns the function into a generator, allowing the caller to process one line at a time.
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2024-11-27 17:06:06,192 : INFO : reading file reviews_data.txt.gz...this may take a while
2024-11-27 17:06:06,202 : INFO : read 0 reviews
2024-11-27 17:06:07,276 : INFO : read 10000 reviews
2024-11-27 17:06:08,343 : INFO : read 20000 reviews
2024-11-27 17:06:09,560 : INFO : read 30000 reviews
2024-11-27 17:06:10,707 : INFO : read 40000 reviews
2024-11-27 17:06:11,959 : INFO : read 50000 reviews
2024-11-27 17:06:13,186 : INFO : read 60000 reviews
2024-11-27 17:06:14,226 : INFO : read 70000 reviews
2024-11-27 17:06:15,320 : INFO : read 80000 reviews
2024-11-27 17:06:16,352 : INFO : read 90000 reviews
2024-11-27 17:06:17,340 : INFO : read 100000 reviews
2024-11-27 17:06:18,345 : INFO : read 110000 reviews
2024-11-27 17:06:19,319 : INFO : read 120000 reviews
2024-11-27 17:06:20,322 : INFO : read 130000 reviews
2024-11-27 17:06:21,398 : INFO : read 140000 reviews
2024-11-27 17:06:22,377 : INFO : read 150000 reviews
2024-11-27 17:06:23,399 : INFO : read 160000 reviews
2024-11-27 17:06:24,725

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [5]:
# 150 features vector, 
# window=10: maximum distance between the target words and context words to consider during training
# workers = 10, 10 threads
# min_count=2, Ignores words that appear fewer than 2 times in the corpus
model = gensim.models.Word2Vec (documents, window=10, min_count=2, workers=10, vector_size=150)
model.train(documents,total_examples=len(documents),epochs=10)

2024-11-27 17:37:51,644 : INFO : collecting all words and their counts
2024-11-27 17:37:51,645 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-11-27 17:37:52,346 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2024-11-27 17:37:52,525 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2024-11-27 17:37:52,698 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2024-11-27 17:37:52,856 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2024-11-27 17:37:53,036 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2024-11-27 17:37:53,206 : INFO : PROGRESS: at sentence #60000, processed 11013728 words, keeping 76788 word types
2024-11-27 17:37:53,350 : INFO : PROGRESS: at sentence #70000, processed 12637530 words, keeping 83201 word types
2024-11-27 17:37:53,488 : INFO : PROG

(303484248, 415193600)

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [6]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


[('filthy', 0.899747908115387),
 ('stained', 0.8173495531082153),
 ('grubby', 0.8132308125495911),
 ('dusty', 0.8078852891921997),
 ('unclean', 0.7988189458847046),
 ('smelly', 0.7799620032310486),
 ('grimy', 0.772834300994873),
 ('gross', 0.7715904116630554),
 ('dingy', 0.7679505944252014),
 ('soiled', 0.754100501537323)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [7]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9302603006362915),
 ('cordial', 0.8585420250892639),
 ('friendly', 0.85205078125),
 ('curteous', 0.8489306569099426),
 ('attentive', 0.8277871608734131),
 ('professional', 0.8228940367698669)]

In [53]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('canada', 0.6603403091430664),
 ('germany', 0.6510637998580933),
 ('spain', 0.6431018114089966),
 ('barcelona', 0.61174076795578),
 ('mexico', 0.6070996522903442),
 ('rome', 0.6065913438796997)]

In [8]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.8324597477912903),
 ('amazed', 0.7985029816627502),
 ('appalled', 0.7876924872398376),
 ('astonished', 0.781569242477417),
 ('stunned', 0.7780076861381531),
 ('dismayed', 0.7765069007873535)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [55]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.7086508274078369),
 ('blanket', 0.7016597390174866),
 ('mattress', 0.7002605199813843),
 ('quilt', 0.6868821978569031),
 ('matress', 0.6777950525283813),
 ('pillowcase', 0.6413239240646362),
 ('sheets', 0.6382123827934265),
 ('foam', 0.6322235465049744),
 ('pillows', 0.6320573687553406),
 ('comforter', 0.5972476601600647)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [9]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.77996206

In [10]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [59]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.25355593501920781

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [11]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [12]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `vector_size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
